In [1]:
pip uninstall autogen-agentchat pyautogen -y

Found existing installation: autogen-agentchat 0.4.9.3
Uninstalling autogen-agentchat-0.4.9.3:
  Successfully uninstalled autogen-agentchat-0.4.9.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install autogen-agentchat

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show autogen-agentchat

Name: autogen-agentchat
Version: 0.4.9.3
Summary: AutoGen agents and teams library
Home-page: 
Author: 
Author-email: 
License: MIT License
        
            Copyright (c) Microsoft Corporation.
        
            Permission is hereby granted, free of charge, to any person obtaining a copy
            of this software and associated documentation files (the "Software"), to deal
            in the Software without restriction, including without limitation the rights
            to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
            copies of the Software, and to permit persons to whom the Software is
            furnished to do so, subject to the following conditions:
        
            The above copyright notice and this permission notice shall be included in all
            copies or substantial portions of the Software.
        
            THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
            IMPLIED, INCLUDING B

In [17]:
import os
import json
import asyncio
import re
import datetime
from typing import Sequence, Union, Optional

# --- Gerekli Kütüphaneleri İçe Aktarma (GÜNCELLENMİŞ YOLLAR) ---
# import from autogen yok! Doğrudan alt modüllerden import ediyoruz.


# Ajanlar, Takımlar, Mesajlar vb. (autogen_agentchat altında)
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import (
    AgentEvent, ChatMessage, TextMessage, ToolCallRequestEvent,
    ToolCallExecutionEvent, ToolCallSummaryMessage
)

# LLM İstemcisi (autogen_ext altında)
from autogen_ext.models.openai import OpenAIChatCompletionClient

# UI (opsiyonel, konsol çıktısı için) (autogen_agentchat altında)
from autogen_agentchat.ui import Console

# Diğer gerekli kütüphaneler
from duckduckgo_search import DDGS

# --- Yer Tutucu Fonksiyonlar (Değişiklik Yok) ---

def log_event(event_details: str) -> str:
    log_file = "assistant_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] {event_details}\n")
        return f"Event logged successfully to {log_file}: {event_details}"
    except Exception as e:
        print(f"Error logging event: {e}")
        return f"Error logging event: {e}"

def perform_web_search(query: str, max_results: int = 3) -> str:
    print(f"[Web Arama] DuckDuckGo ile aranıyor: '{query}' (Maks {max_results} sonuç)")
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))
            if not results:
                return f"No search results found for '{query}'."
            formatted_results = "\n\n".join([f"Title: {res.get('title', 'N/A')}\nLink: {res.get('href', 'N/A')}\nSnippet: {res.get('body', 'N/A')}" for res in results])
            return f"Search results for '{query}':\n{formatted_results}"
    except Exception as e:
        print(f"[Hata] Web araması sırasında hata oluştu: {e}")
        return f"Error during web search for '{query}': {e}"

def save_email_draft(to: str, subject: str, body: str) -> str:
    draft_file = "email_drafts.txt"
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    draft_content = f"""--- Draft Saved: {timestamp} ---
To: {to}
Subject: {subject}
--- Body ---
{body}
--- End Draft ---
"""
    print(f"[E-posta Taslağı] Taslak '{draft_file}' dosyasına kaydediliyor...")
    try:
        with open(draft_file, "a", encoding="utf-8") as f:
            f.write(draft_content + "\n\n")
        return f"Email draft for {to} with subject '{subject}' saved to {draft_file}."
    except Exception as e:
        print(f"[Hata] E-posta taslağı kaydedilirken hata oluştu: {e}")
        return f"Error saving email draft: {e}"

def add_todo_item(task: str) -> str:
    todo_file = "todo_list.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    print(f"[Yapılacaklar Listesi] Görev '{todo_file}' dosyasına ekleniyor: '{task}'")
    try:
        with open(todo_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] - {task}\n")
        return f"Task '{task}' added to the To-Do list ({todo_file})."
    except Exception as e:
        print(f"[Hata] Yapılacaklar listesine eklenirken hata oluştu: {e}")
        return f"Error adding task to To-Do list: {e}"

# --- Yapılandırma (LLM İstemcisi Oluşturma - config_list_from_json OLMADAN) ---

config_list_path = "OAI_CONFIG_LIST.json" # JSON dosyanızın yolu
try:
    # JSON dosyasını manuel olarak oku
    with open(config_list_path, 'r', encoding='utf-8') as f:
        config_data = json.load(f)

    # Genellikle JSON bir liste içerir, ilk öğeyi alalım
    if isinstance(config_data, list) and len(config_data) > 0:
        llm_config_params = config_data[0]
    elif isinstance(config_data, dict): # Eğer liste değilse, doğrudan sözlük olabilir
         llm_config_params = config_data
    else:
        raise ValueError(f"{config_list_path} dosyasında geçerli yapılandırma bulunamadı.")

    # Gerekli parametreleri al
    # API anahtarı için önce ortam değişkenini kontrol et, sonra JSON'u
    api_key = os.getenv("OPENAI_API_KEY", llm_config_params.get("api_key"))
    model_name = llm_config_params.get("model", "gpt-4o") # Varsayılan model veya JSON'dan gelen

    if not api_key:
        raise ValueError("OpenAI API anahtarı ne JSON dosyasında ne de OPENAI_API_KEY ortam değişkeninde bulunamadı.")

    # OpenAIChatCompletionClient istemcisini oluştur
    model_client = OpenAIChatCompletionClient(
        model=model_name,
        api_key=api_key,
        # Gerekirse diğer parametreleri ekleyin (örn. base_url, api_version llm_config_params'dan alınabilir)
        # temperature=0.5 # Sıcaklık gibi ayarlar run metodunda verilebilir
    )
    print(f"OpenAI İstemcisi '{model_name}' modeli için başarıyla oluşturuldu.")

except FileNotFoundError:
    print(f"Hata: Yapılandırma dosyası bulunamadı: {config_list_path}")
    exit()
except (json.JSONDecodeError, ValueError, KeyError) as e: # KeyError eklendi
    print(f"Hata: Yapılandırma hatası ({config_list_path}): {e}")
    exit()
except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")
    exit()


# --- Ajan Tanımları (model_client ve description ile) ---
# Ajan tanımları önceki güncellenmiş haliyle aynı kalabilir, sadece importlar değişti.

# Orkestratör / Yönetici Ajan
orchestrator = AssistantAgent(
    name="Orchestrator",
    model_client=model_client,
    system_message="""You are the Orchestrator/Supervisor Agent managing a multi-agent team. Your primary responsibility is to meticulously follow a plan to fulfill the user's request.
1.  Receive the initial user query.
2.  Delegate to 'Input_Preprocessor' to clean the query. Receive the cleaned query.
3.  Delegate to 'Intent_Task_Router' to get a step-by-step plan. Receive the plan.
4.  **Execute the plan STRICTLY step-by-step:**
    a.  Identify the **NEXT** agent required by the plan (e.g., Web_Search_Agent, Summarization_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent, Analytics_Logging_Agent).
    b.  Instruct that specific agent clearly, providing ONLY the necessary input derived from the user query or previous step's result. If a tool call is needed, specify the tool name and arguments.
    c.  **Wait** for that agent to complete its task and report its result back to you.
    d.  **Verify** the agent provided the expected output for its role. **IGNORE** any extra work, summaries, or suggestions the agent might add beyond its core task result.
    e.  Use the CORE result from the previous step as input for the **NEXT** agent in the plan.
    f.  Repeat steps a-e until **ALL** steps in the plan received from 'Intent_Task_Router' are successfully completed. The typical flow for the current request might be: Web Search -> Summarize -> Draft & Save Email -> Add ToDo -> Log Events.
5.  **DO NOT** skip steps in the plan. **DO NOT** let agents perform tasks outside their defined role or the current step.
6.  If clarification is needed at any step, instruct 'Follow_Up_Clarification_Agent' to formulate a question, then present that question (usually via User_Proxy).
7.  After all planned steps are finished, provide a final concise summary *if appropriate* based on the initial request.
8.  **FINALLY**, conclude the entire process by responding with the single word "TERMINATE".""",
    description="Manages the workflow strictly based on a plan, delegates tasks sequentially, processes intermediate results, ensures role adherence, and completes all steps before terminating."
)

# Kullanıcı Proxy Ajanı
user_proxy = UserProxyAgent(
    name="User_Proxy",
    description="Represents the end-user who initiates the request. Does not perform tasks itself, but receives the final outcome. May provide input if clarification is explicitly requested."
)

# Girdi Ön İşleme Ajanı
input_preprocessor = AssistantAgent(
    name="Input_Preprocessor",
    model_client=model_client,
    system_message="""You are the Input Preprocessing Agent.
1.  Receive the raw user query from the Orchestrator.
2.  Clean grammar, spelling, and identify key entities.
3.  **Your ONLY output MUST be the cleaned query itself, prefixed with 'Cleaned Query:'.**
    Example: 'Cleaned Query: Search the web for AutoGen benefits, summarize them, email Alex, and add a todo.'
4.  DO NOT add conversational filler, explanations, or confirmations.""",
    description="Cleans and preprocesses the initial user query, returning only the cleaned text prefixed with 'Cleaned Query:'."
)

# Niyet ve Görev Yönlendirme Ajanı
intent_task_router = AssistantAgent(
    name="Intent_Task_Router",
    model_client=model_client,
    system_message="""You are the Intent & Task Routing Agent.
1.  Receive the preprocessed query from the Orchestrator.
2.  Determine user intent(s).
3.  Create a sequential plan outlining the agents needed and their inputs/tasks. Use the official agent names (Web_Search_Agent, Summarization_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent, Analytics_Logging_Agent etc.).
4.  **Your ONLY output MUST be the plan itself, prefixed with 'Plan:'.**
    Example: 'Plan: 1. Web_Search_Agent (query='benefits of Autogen framework for multi-agent systems'). 2. Summarization_Agent (input=search_results, format='3 bullets'). 3. Email_Composition_Sentiment_Agent (to='alex@example.com', subject='Autogen Benefits', context=summary_from_step_2). 4. To_Do_List_Agent (task='Prepare presentation on Autogen'). 5. Analytics_Logging_Agent (details='User request processed').'
5.  DO NOT execute any tasks yourself. DO NOT add conversational filler.""",
    description="Analyzes the query, creates a step-by-step execution plan with agent names and inputs, returning only the plan prefixed with 'Plan:'."
)

# --- Uzmanlaşmış Görev Ajanları (Tools ile) ---

# Web Arama Ajanı
web_search_agent = AssistantAgent(
    name="Web_Search_Agent",
    model_client=model_client,
    tools=[perform_web_search],
    reflect_on_tool_use=True, # Tool sonucunu anlatan bir metin üretebilir ama fazlasını değil
    system_message="""You are the Web Search Agent. Your **ONLY** function is to execute web searches using the 'perform_web_search' tool when instructed by the Orchestrator.
1.  Receive a search query instruction from the Orchestrator.
2.  Call the 'perform_web_search' tool with the exact query and parameters provided.
3.  **Your ONLY output MUST be the raw search result string returned by the tool.** If `reflect_on_tool_use` is enabled, you may preface it with a very brief statement like "Here are the search results:", but nothing more.
4.  **DO NOT** summarize, analyze, interpret the results, draft emails, add todos, or ask follow-up questions. Report ONLY the tool's direct output.""",
    description="STRICTLY performs web searches using 'perform_web_search' tool and reports ONLY the raw tool output."
)

# Özetleme Ajanı
summarization_agent = AssistantAgent(
    name="Summarization_Agent",
    model_client=model_client,
    system_message="""You are the Summarization Agent. Your **ONLY** task is to summarize text provided by the Orchestrator according to specific instructions (e.g., length, format).
1.  Receive text and summarization instructions from the Orchestrator.
2.  Generate the summary based strictly on the input text and instructions.
3.  **Your ONLY output MUST be the summary itself.**
4.  DO NOT perform web searches, draft emails, or call any tools.""",
    description="Summarizes text provided by the Orchestrator based on instructions, returning ONLY the summary."
)

# Zamanlama ve Takvim Ajanı (Simülasyonlu)
scheduling_calendar_agent = AssistantAgent(
    name="Scheduling_Calendar_Agent",
    model_client=model_client,
    system_message="""You are the Scheduling & Calendar Agent. Your task is to **simulate** calendar operations based on instructions from the Orchestrator.
1.  Receive a scheduling instruction (e.g., 'schedule a meeting', 'find free time') from the Orchestrator, including necessary details (time, participants, subject).
2.  **You DO NOT have access to a real calendar.** Your ONLY action is to formulate a response indicating the *simulated* outcome.
3.  Respond with a clear message stating what was simulated and whether it was successful (hypothetically).
    Example responses:
    - 'Simulated: Meeting "Project Alpha Kickoff" scheduled for tomorrow at 10 AM with Alex and Bob.'
    - 'Simulated: Conflict found for the requested time slot. Cannot schedule.'
    - 'Simulated: Found available time slot: Wednesday at 2 PM.'
4.  DO NOT ask for confirmation or perform other tasks. Report ONLY the simulation result.""",
    description="Simulates calendar operations (scheduling events, finding times) based on Orchestrator's instructions and reports the simulated outcome."
)

# E-posta Oluşturma ve Duygu Analizi Ajanı
email_sentiment_agent = AssistantAgent(
    name="Email_Composition_Sentiment_Agent",
    model_client=model_client,
    tools=[save_email_draft],
    reflect_on_tool_use=True, # Tool sonucunu yazdırabilir
    system_message="""You are the Email Composition & Sentiment Analysis Agent.
*   **IF INSTRUCTED TO DRAFT EMAIL by Orchestrator:**
    1.  Receive context (e.g., summary) and recipient details (to, subject) from the Orchestrator.
    2.  Generate *only* the email body content based on the provided context.
    3.  **WAIT** for the Orchestrator's explicit instruction to call the 'save_email_draft' tool.
    4.  When instructed, call 'save_email_draft' with the 'to', 'subject', and the generated 'body' provided by the Orchestrator.
    5.  Report the exact result message returned by the 'save_email_draft' tool.
*   **IF INSTRUCTED TO ANALYZE SENTIMENT by Orchestrator:**
    1.  Receive text from the Orchestrator.
    2.  Analyze the sentiment (positive, negative, neutral).
    3.  Report **only** the sentiment analysis result.
*   **DO NOT** perform web searches, add todos, or initiate tool calls without explicit instruction from the Orchestrator for *that specific step*. DO NOT ask for confirmation to send.""",
    description="Drafts email BODY based on context OR analyzes sentiment when instructed. Calls 'save_email_draft' tool ONLY when explicitly told to by Orchestrator."
)

# Yapılacaklar Listesi Ajanı
todo_list_agent = AssistantAgent(
    name="To_Do_List_Agent",
    model_client=model_client,
    tools=[add_todo_item],
    reflect_on_tool_use=True, # Tool sonucunu yazdırabilir
    system_message="""You are the To-Do List Agent. Your **ONLY** task is to add items to the to-do list using the 'add_todo_item' tool when instructed by the Orchestrator.
1.  Receive the exact task description from the Orchestrator.
2.  Call the 'add_todo_item' tool with that exact task description.
3.  Report **only** the confirmation message returned by the tool.""",
    description="Adds tasks to the to-do list using 'add_todo_item' tool ONLY when instructed, reporting only the tool's result."
)

# --- Destekleyici Ajanlar ---

# Takip ve Açıklama Ajanı
follow_up_clarification_agent = AssistantAgent(
    name="Follow_Up_Clarification_Agent",
    model_client=model_client,
    system_message="""You are the Follow-Up & Clarification Agent.
1.  Receive instructions from the Orchestrator when clarification is needed.
2.  Formulate a concise question for the user based *only* on the Orchestrator's request.
3.  **Your ONLY output MUST be the question itself.**
    Example: 'What is the email address for Alex?'
4.  DO NOT add conversational filler.""",
    description="Formulates clarifying questions based ONLY on Orchestrator's request, returning ONLY the question."
)

# Analiz ve Loglama Ajanı
analytics_logging_agent = AssistantAgent(
    name="Analytics_Logging_Agent",
    model_client=model_client,
    tools=[log_event],
    reflect_on_tool_use=True, # Tool sonucunu yazdırabilir
    system_message="""You are the Analytics & Logging Agent. Your **ONLY** task is to log events using the 'log_event' tool when instructed by the Orchestrator.
1.  Receive the exact event details to log from the Orchestrator.
2.  Call the 'log_event' tool with those exact details.
3.  Report **only** the success message returned by the tool.""",
    description="Logs specific events using the 'log_event' tool ONLY when instructed, reporting only the tool's result."
)

# --- Grup Sohbeti Kurulumu (SelectorGroupChat ile) ---

agents = [
    user_proxy, orchestrator, input_preprocessor, intent_task_router,
    web_search_agent, summarization_agent, scheduling_calendar_agent,
    email_sentiment_agent, todo_list_agent, follow_up_clarification_agent,
    analytics_logging_agent,
]

# Sonlandırma Koşulları
text_mention_termination = TextMentionTermination(
    text="TERMINATE",
    sources=[orchestrator.name]
)
max_messages_termination = MaxMessageTermination(max_messages=50)

# İki koşulu OR (|) operatörü ile birleştir (GÜNCELLENDİ)
combined_termination_condition = text_mention_termination | max_messages_termination

selector_prompt = """You are overseeing a conversation between agents working towards a user goal, following a plan.
Available roles:
{roles}

Conversation history:
{history}

Participants: {participants}.
The **Orchestrator** agent MUST manage the workflow step-by-step according to the initial plan. It delegates one task at a time and processes the result before delegating the next step.

Read the history carefully. Select the *next* agent to speak based on these STRICT rules:
1.  If the Orchestrator just gave a clear instruction/delegation to a *specific* agent, **SELECT THAT AGENT**.
2.  If *any* other agent (e.g., Web_Search_Agent, Summarization_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent) just finished its task and reported its result, **SELECT THE Orchestrator**. The Orchestrator MUST process the result and decide the next step according to the plan. **DO NOT** select another specialized agent immediately after one has reported.
3.  If the conversation seems stuck, unclear, or needs the next step from the plan, **SELECT THE Orchestrator**.
4.  If the Orchestrator instructed the Follow_Up_Clarification_Agent, select it. If Follow_Up_Clarification_Agent just spoke, select Orchestrator.
5.  If the User_Proxy just provided the initial request, select Orchestrator (or Input_Preprocessor if that's the first step).

**Only return the exact name of the single agent selected.** No extra text.
"""


# SelectorGroupChat Takımını Oluştur (GÜNCELLENDİ)
team = SelectorGroupChat(
    participants=agents,
    model_client=model_client, # LLM bu istemciyi kullanarak seçim yapacak
    termination_condition=combined_termination_condition,
    # selector_func=custom_selector_func, # KALDIRILDI
    selector_prompt=selector_prompt,     # EKLENDİ - LLM'e rehberlik eder
    allow_repeated_speaker=True,       # LLM aynı ajanı (örn. Orkestratör) tekrar seçebilir
)

# --- Sohbeti Başlatma (Asenkron) ---

user_query = (
    "Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, "
    "summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) "
    "explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list."
)

print(f"--- Sohbet Başlatılıyor --- \nSorgu: {user_query}\n{'='*30}")

async def run_team_chat_stream():
    # team.run() yerine team.run_stream() kullanıyoruz
    # Console, akıştaki mesajları otomatik olarak yazdıracak
    stream = team.run_stream(task=user_query)
    final_result = await Console(stream) # Console akışı işler ve son TaskResult'ı döndürür

    print(f"\n{'='*30}\n--- Sohbet Sona Erdi ---")
    # final_result None olabilir eğer stream'de hata olursa, kontrol edelim
    if final_result:
        print(f"Sonlanma Nedeni: {final_result.stop_reason}")
        print(f"Toplam Mesaj Sayısı: {len(final_result.messages)}")
        messages = final_result.messages
    else:
        print("Sohbet sonucu alınamadı.")
        messages = [] # Hata durumunda boş liste


    # --- Son Çıktıyı İşleme (Önceki gibi ama 'messages' listesini kullanır) ---
    final_output = "Nihai çıktı işlenemedi veya bulunamadı."
    log_summary = []

    if messages:
        # Mesaj işleme döngüsü aynı kalır...
        for msg in reversed(messages):
            speaker_name: Optional[str] = None
            speaker_ref = getattr(msg, 'source', None)
            if hasattr(speaker_ref, 'name'):
                speaker_name = speaker_ref.name
            elif isinstance(speaker_ref, str):
                speaker_name = speaker_ref

            content_str = str(getattr(msg, 'content', ''))

            if isinstance(msg, TextMessage) and speaker_name == orchestrator.name:
                 if content_str.strip() and "TERMINATE" not in content_str:
                     final_output = f"(Son Orkestratör mesajı):\n{content_str.strip()}"
                     # break # Logları toplamak için devam et
                 elif content_str.strip() and "TERMINATE" in content_str and final_output.startswith("Nihai çıktı"):
                      final_output = f"(TERMINATE içeren son Orkestratör mesajı):\n{content_str.replace('TERMINATE','').strip()}"

            elif speaker_name == analytics_logging_agent.name:
                 if "logged successfully" in content_str:
                      log_summary.append(content_str)


    print(f"\n--- Nihai Çıktı ---")
    print(final_output)

    print("\n--- Loglanan Olaylar Özeti ---")
    if log_summary:
        for log_entry in reversed(log_summary): # Kronolojik sıra
            print(f"- {log_entry}")
    else:
        print("Loglanan olay bulunamadı.")

    if os.path.exists("assistant_log.txt"):
        print("\nLog detayları 'assistant_log.txt' dosyasına yazıldı.")

# Asenkron sohbeti çalıştır
if __name__ == "__main__":
    try:
        # run_team_chat yerine run_team_chat_stream'i çağırıyoruz
        await run_team_chat_stream() # Jupyter/IPython için await
    except Exception as e:
        print(f"\n--- Çalışma Zamanı Hatası ---")
        import traceback
        traceback.print_exc()

OpenAI İstemcisi 'gpt-4-1106-preview' modeli için başarıyla oluşturuldu.
--- Sohbet Başlatılıyor --- 
Sorgu: Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list.
---------- user ----------
Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list.
---------- Input_Preprocessor ----------
Cleaned Query: Search the web for the main benefits of using the Autogen framework for multi-agent systems, summarize the key points into three bullet points, draft an email to my colleague Alex (alex@example.com) explaining these be